# Year 2023 Day 10


The main goal here is to show the different steps in the labyrinth resolution.

Experiment with the Observable Plot's Text mark, to add a Text mark over a Raster mark ((see example in https://observablehq.com/plot/marks/raster))


In [ ]:
from typing import Any

import pandas as pd
import xarray as xr
from pyobsplot import Plot

from advent_of_code.common.common import get_example_inputs_file_contents
from advent_of_code.visualization.observable_plot import (
    build_base_xarray_plot,
)
from advent_of_code.y_2023.problem_202310 import (
    AdventOfCodeProblem202310,
    get_pipe_to_pattern_mapping,
    render_2d_array_to_text,
)

In [ ]:
problem = AdventOfCodeProblem202310(config_accumulate_outputs=True)
problem

## Visualization Code

The following function enriches the base xarray visualization function with Observable Plot, by adding the possibility to add a text-representation of the array on the plot.

This is useful for a combined visualization of a raster and its values.

In [ ]:
def visualize_puzzle_input_202310(
    xda: xr.DataArray,
    *,
    text_xda: xr.DataArray | None = None,
    **kwargs: Any,
):
    def callback(marks: list[Any]) -> None:
        if text_xda is not None:
            df = text_xda.stack(z=("row", "col")).to_pandas()
            data = df.values.tolist()
            rows = [idx[0] + 0.5 for idx in df.index]
            cols = [idx[1] + 0.5 for idx in df.index]
            marks.append(
                Plot.text(  # type:ignore
                    data,
                    {
                        "text": Plot.identity,  # type: ignore
                        "x": cols,
                        "y": rows,
                        "color": "white",
                        "fontSize": 38 * kwargs["width"] / 600 * 13 / xda["col"].size,
                    },
                )
            )

    return build_base_xarray_plot(
        xda, callback, margin=0, do_convert_ascii_array_to_uint8=False, **kwargs
    )

## With Example Inputs

### Example input from part 2 (`EXAMPLE_INPUT_PART_2_1_1a`)

#### Problem solving explanation

In the following, we visualize step by step the data structures helping to solve the problem.

In [ ]:
example_inputs_2023 = get_example_inputs_file_contents(2023)["test_problem_202310"]
list(example_inputs_2023.keys())

In [ ]:
puzzle_input = problem.parse_text_input(
    example_inputs_2023["EXAMPLE_INPUT_PART_2_1_1a"]
)
maze = puzzle_input
minimum_distances = problem.compute_minimum_distances(maze)
result, outputs_accumulator = problem.compute_tiles_enclosed_by_loop_part_2(
    maze, minimum_distances
)
display(pd.Series(outputs_accumulator.keys()))

print("Text Input")
print(render_2d_array_to_text(outputs_accumulator["maze"]))

First, the main loop must be identified. The heatmap of minimal distance from starting point can be used for this. 

In [ ]:
min_dist_xda = xr.DataArray(
    outputs_accumulator["minimum_distances"], dims=["row", "col"]
)

visualize_puzzle_input_202310(
    min_dist_xda,
    color=dict(legend=True, scheme="magma"),
    title="Minimum Distances",
    width=200,
)

All pipes are identified by strictly positive integers ; the starting position has a special value of $-1$, and empty space is denoted by $0$.

In [ ]:
min_dist_text_xda = xr.DataArray(
    outputs_accumulator["minimum_distances"], dims=("row", "col")
)
visualize_puzzle_input_202310(
    min_dist_xda,
    text_xda=min_dist_text_xda,
    color=dict(legend=True, scheme="magma"),
    title="Minimum Distances (with values)",
    width=200,
)

The main loop can now be identified using the minimum distances.

In [ ]:
main_loop_xda = xr.DataArray(outputs_accumulator["main_loop"], dims=["row", "col"])
text_xda = xr.DataArray(outputs_accumulator["maze"], dims=("row", "col"))

visualize_puzzle_input_202310(
    main_loop_xda,
    text_xda=text_xda,
    color=dict(legend=True, scheme="Observable10"),
    title="(Before) Main Loop",
    width=200,
)

The trick is to replace in pipe symbol of the Main loop by a 3x3 macro-pixel materializing this symbol:


In [ ]:
mapping = get_pipe_to_pattern_mapping()
mapping_xda = xr.Dataset(
    {k: xr.DataArray(v, dims=("row", "col")) for k, v in mapping.items()}
).to_array(dim="pipe")
mapping_xda.plot(y="row", x="col", col="pipe", yincrease=False)

In [ ]:
raster_3x_xda = xr.DataArray(outputs_accumulator["raster_3x"], dims=["row", "col"])
visualize_puzzle_input_202310(
    raster_3x_xda,
    color=dict(legend=True, scheme="viridis"),
    title="(After) Raster 3x",
    width=200,
)

Then, this new raster truly materializes the pipe network.


To find the inside area is now easy: use a [Flood Fill](https://en.wikipedia.org/wiki/Flood_fill) algorithm, for instance implemented by scipy: [skimage.morphology.flood_fill](https://scikit-image.org/docs/stable/api/skimage.morphology.html#skimage.morphology.flood_fill).


In [ ]:
filled_xda = xr.DataArray(outputs_accumulator["filled"], dims=["row", "col"])
visualize_puzzle_input_202310(
    filled_xda,
    color=dict(legend=True, scheme="Observable10"),
    title="Filled (flood fill)",
    width=200,
)

The final trick is to go back to the original raster, knowing which pixels are filled.
A pixel is considered as filled if its corresponding 3x3 macro-pixel is entirely filled. This excludes the macro-pixels representing pipe from the count.


In [ ]:
coarsened_img_xda = xr.DataArray(
    outputs_accumulator["coarsened_img"], dims=["row", "col"]
)
visualize_puzzle_input_202310(
    coarsened_img_xda == 255,
    color=dict(legend=True, scheme="viridis"),
    title="Coarsened 3x",
    width=200,
)

#### Recapitulation

In the following, we display all the representations of the successive intermediate data structures helping to solve the problem.

In [ ]:
def visualize_puzzle_input(outputs_accumulator, with_text: bool = True, **kwargs):
    min_dist_xda = xr.DataArray(
        outputs_accumulator["minimum_distances"], dims=["row", "col"]
    )
    visualize_puzzle_input_202310(
        min_dist_xda,
        color=dict(legend=True, scheme="magma"),
        title="Minimum Distances",
        **kwargs,
    )

    min_dist_text_xda = xr.DataArray(
        outputs_accumulator["minimum_distances"], dims=("row", "col")
    )
    visualize_puzzle_input_202310(
        min_dist_xda,
        text_xda=min_dist_text_xda,
        color=dict(legend=True, scheme="magma"),
        title="Minimum Distances (with values)",
        **kwargs,
    )

    main_loop_xda = xr.DataArray(outputs_accumulator["main_loop"], dims=["row", "col"])
    if with_text:
        text_xda = xr.DataArray(outputs_accumulator["maze"], dims=("row", "col"))
    else:
        text_xda = None
    visualize_puzzle_input_202310(
        main_loop_xda,
        text_xda=text_xda,
        color=dict(legend=True, scheme="Observable10"),
        title="Main Loop",
        **kwargs,
    )

    raster_3x_xda = xr.DataArray(outputs_accumulator["raster_3x"], dims=["row", "col"])
    visualize_puzzle_input_202310(
        raster_3x_xda,
        color=dict(legend=True, scheme="viridis"),
        title="Raster 3x",
        **kwargs,
    )

    filled_xda = xr.DataArray(outputs_accumulator["filled"], dims=["row", "col"])
    visualize_puzzle_input_202310(
        filled_xda,
        color=dict(legend=True, scheme="Observable10"),
        title="Filled (flood fill)",
        **kwargs,
    )

    coarsened_img_xda = xr.DataArray(
        outputs_accumulator["coarsened_img"], dims=["row", "col"]
    )
    visualize_puzzle_input_202310(
        coarsened_img_xda == 255,
        color=dict(legend=True, scheme="viridis"),
        title="Coarsened 3x",
        **kwargs,
    )

### Example input from part 1 (`EXAMPLE_INPUT_2_1`)


Same visualization procedure.


In [ ]:
maze = problem.parse_text_input(example_inputs_2023["EXAMPLE_INPUT_2_1"])
result, outputs_accumulator = problem.compute_tiles_enclosed_by_loop_part_2(
    maze, problem.compute_minimum_distances(maze)
)

print("Text Input")
print(render_2d_array_to_text(outputs_accumulator["maze"]))

visualize_puzzle_input(outputs_accumulator, width=200)

In [ ]:
maze = problem.parse_text_input(example_inputs_2023["EXAMPLE_INPUT_PART_2_1_1a"])
result, outputs_accumulator = problem.compute_tiles_enclosed_by_loop_part_2(
    maze, problem.compute_minimum_distances(maze)
)

print("Text Input")
print(render_2d_array_to_text(outputs_accumulator["maze"]))

visualize_puzzle_input(outputs_accumulator, width=200)

## With Actual Input

Same visualization procedure.


In [ ]:
def visualize_actual_input(problem: AdventOfCodeProblem202310):
    maze = problem.parse_input_text_file()

    minimum_distances = problem.compute_minimum_distances(maze)
    result, outputs_accumulator = problem.compute_tiles_enclosed_by_loop_part_2(
        maze, minimum_distances
    )

    visualize_puzzle_input(
        outputs_accumulator,
        width=maze.shape[0] * 3,
        height=maze.shape[0] * 3,
        with_text=False,
    )


visualize_actual_input(problem)